In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, Dropout, BatchNormalization, Convolution2D, Activation, MaxPooling2D, Flatten, AveragePooling2D, Input, GaussianNoise
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.models import load_model, Sequential
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import time
import cv2
import random
import math
from skimage.transform import resize
from numpy import rollaxis
from IPython.display import FileLink
from scipy.ndimage.interpolation import rotate
from scipy import ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

# !pip install zip_files

%matplotlib inline



In [3]:
x_train = np.load("x_train_2.npz")["arr_0"]
y_train = np.load("y_train_2.npz")["arr_0"]

In [4]:
# # from google.colab.patches import cv2_imshow

# for i in range(x_train.shape[0]):
#     if random.randint(0,15) < 5:
#         print(i)
        
#         rand_num = random.randint(0,3)
#         if rand_num == 0:
#             next_image = x_train[i,:,:,:]
#             next_label = y_train[i,:,:]

#             zeros = np.zeros((244,244,3))
#             shift = random.randint(60,100)
#               # print(x_shift)
#               # print(zeros[x_shift:,:,:].shape)
#               # print(next_image[:next_image.shape[0] - x_shift,:,:].shape)
#             zeros[shift:,:,:] = next_image[:next_image.shape[0] - shift,:,:]
#             next_label[:,1] += shift
            
#         elif rand_num == 1:
#             next_image = x_train[i,:,:,:]
#             next_label = y_train[i,:,:]

#             zeros = np.zeros((244,244,3))
#             shift = random.randint(-100,-60)
#               # print(x_shift)
#               # print(zeros[x_shift:,:,:].shape)
#               # print(next_image[:next_image.shape[0] - x_shift,:,:].shape)
#             zeros[:shift,:,:] = next_image[-shift:,:,:]
#             next_label[:,1] += shift

#         elif rand_num == 2:
#             next_image = x_train[i,:,:,:]
#             next_label = y_train[i,:,:]

#             zeros = np.zeros((244,244,3))
#             shift = random.randint(60,100)
#               # print(x_shift)
#               # print(zeros[x_shift:,:,:].shape)
#               # print(next_image[:next_image.shape[0] - x_shift,:,:].shape)
#             zeros[:,shift:,:] = next_image[:,:next_image.shape[1] - shift,:]
#             next_label[:,0] += shift
            
#         elif rand_num == 3:
#             next_image = x_train[i,:,:,:]
#             next_label = y_train[i,:,:]

#             zeros = np.zeros((244,244,3))
#             shift = random.randint(-100,-60)
#               # print(x_shift)
#               # print(zeros[x_shift:,:,:].shape)
#               # print(next_image[:next_image.shape[0] - x_shift,:,:].shape)
#             zeros[:,:shift,:] = next_image[:,-shift:,:]
#             next_label[:,0] += shift
        
        
# #         print(zeros.shape)
# #         for point in next_label:
# #             zeros = cv2.circle(zeros, tuple([int(x) for x in point[:2]]), radius=3, color=(0, 0, 255), thickness=-1)

# #         cv2.imshow("window",zeros*255)
        

#         zeros = np.reshape(zeros, (1, 244, 244, 3))
#         next_label = np.reshape(next_label, (1, 68, 3))

#         x_train = np.vstack((x_train, zeros))
#         y_train = np.vstack((y_train, next_label))
    
# #     print(next_label)



    

In [5]:
# np.savez("x_train_2.npz", x_train)
# np.savez("y_train_2.npz", y_train)

In [6]:
# output_dims = 68*3

# print(x.shape)

# ### Model architecture modified from https://towardsdatascience.com/detecting-facial-features-using-deep-learning-2e23c8660a7a

# IMG_SHAPE = (244,244,3)
# base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                                include_top=False, 
#                                                weights='imagenet')
# base_model.trainable = False

# model = tf.keras.Sequential([
#   base_model,
#   Convolution2D(256, 5, strides=(3,3), activation="relu"),
#   MaxPooling2D(),
#   # Convolution2D(128, 5, strides=(3,3), activation="relu"),
#   # MaxPooling2D(),
#   # Convolution2D(128, 3, strides=(3,3), activation="relu"),
#   # Convolution2D(128, 3, strides=(1,1), activation="relu"),
#   Flatten(),
#   Dense(2048, activation="relu"),
#   Dense(204, activation="relu"),
#   Reshape((68, 3))
# ])

# model.compile(optimizer="adam", loss="mean_squared_error")


# # x,y=augment_data(x_train, y_train)
    
    
# #     np.savez_compressed("x_train.npz",x_train)
# #     np.savez_compressed("y_train.npz",y_train)

# filepath = f"/content/drive/MyDrive/facial_model_{time.time()}.h5"

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# # model = load_model(filepath)

# model.fit(
#         x=x,
#         y=y/244,
#         epochs=200000,
#         verbose=1,
#         shuffle=True,
#         steps_per_epoch=16,
#         batch_size=16,
#         validation_data=(x_test,y_test/244),
#         callbacks=callbacks_list)
    

In [7]:

from tensorflow.keras import layers

batch_size = 8

filepath = f"/content/drive/MyDrive/facial_model_{time.time()}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

gray_train = np.dot(x_train[...,:3], [0.299, 0.587, 0.114])

input_img = tf.keras.Input(shape=(244, 244, 1))

x = tf.keras.applications.mobilenet_v2.preprocess_input(input_img)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.UpSampling2D((2, 2))(encoded)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = layers.Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = layers.Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = layers.Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = layers.Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
decoded = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="valid")(x)

autoencoder = tf.keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')



In [8]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244, 244, 1)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 244, 244, 1)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 244, 244, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 244, 244, 256)     2560      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 122, 122, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 128)     295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 128)       0     

In [9]:
# autoencoder.fit(
#         x=gray_train,
#         y=gray_train,
#         epochs=200000,
#         verbose=1,
#         shuffle=True,
#         steps_per_epoch=16,
#         batch_size=16)

In [10]:


backbone = tf.keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(244,244,3)
)

backbone.trainable = False

inputs = layers.Input((244,244,3))
x_ = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x_ = backbone(x_)
x_ = layers.Dropout(0.3)(x_)
x_ = layers.BatchNormalization()(x_)
x_ = layers.SeparableConv2D(
    204, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.BatchNormalization()(x_)
x_ = layers.SeparableConv2D(
    204, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.BatchNormalization()(x_)
x_ = layers.SeparableConv2D(
    204, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.Dropout(0.3)(x_)

x_ = layers.Flatten()(x_)

x_ = layers.Dense(204, activation="sigmoid")(x_)
outputs = layers.Reshape((68,3))(x_)

model = tf.keras.Model(inputs, outputs)

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 244, 244, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 244, 244, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 8, 8, 1280)        2257984   
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 1280)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 1280)        5120      
_________________________________________________________________
separable_conv2d (SeparableC (None, 6, 6, 204)         2728

In [12]:
# filepath = f"facial_model_{time.time()}.h5"

# checkpoint = ModelCheckpoint(filepath, monitor='mean_absolute_error', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# model.compile(loss="mean_absolute_error", optimizer="rmsprop")
# model.fit(x_train, y_train / 244, epochs=100000, batch_size=32, steps_per_epoch=16, callbacks=callbacks_list)

In [13]:
# filepath = f"facial_model_{time.time()}.h5"
# model.save(filepath)

In [14]:
# filepath = f"facial_model_{time.time()}.h5"

# checkpoint = ModelCheckpoint(filepath, monitor='mean_absolute_error', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# model.compile(loss="mean_absolute_error", optimizer="adagrad")
# model.fit(x_train, y_train / 244, epochs=100000, batch_size=32, steps_per_epoch=16, callbacks=callbacks_list)

In [15]:
# filepath = f"facial_model_{time.time()}.h5"
# model.save(filepath)

In [16]:
# filepath = f"facial_model_{time.time()}.h5"

# checkpoint = ModelCheckpoint(filepath, monitor='mean_absolute_percentage_error', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# model.compile(loss="mean_absolute_percentage_error", optimizer="adagrad")
# model.fit(x_train, y_train / 244, epochs=100000, batch_size=32, steps_per_epoch=16, callbacks=callbacks_list)

In [17]:
x, x_test, y, y_test = train_test_split(x_train, y_train, test_size=.20)

print(x.shape)
print(x_train.shape)

print(y.shape)
print(y_train.shape)

(4632, 244, 244, 3)
(5790, 244, 244, 3)
(4632, 68, 3)
(5790, 68, 3)


In [24]:


# backbone = tf.keras.applications.MobileNetV2(
#     weights="imagenet", include_top=False, input_shape=(244,244,3)
# )

# backbone.trainable = False

inputs = layers.Input((244,244,3))
x_ = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
# x_ = backbone(x_)
# x_ = layers.Dropout(0.3)(x_)
# x_ = layers.BatchNormalization()(x_)
x_ = layers.Conv2D(204, kernel_size=3, strides=1, activation="relu")(x_)
x_ = layers.Conv2D(
    32, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.MaxPooling2D()(x_)

x_ = layers.Conv2D(
    32, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.MaxPooling2D()(x_)

x_ = layers.Conv2D(
    64, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.MaxPooling2D()(x_)

x_ = layers.Conv2D(
    64, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.MaxPooling2D()(x_)

x_ = layers.Conv2D(
    204, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.MaxPooling2D()(x_)

x_ = layers.Conv2D(
    204, kernel_size=3, strides=1, activation="relu"
)(x_)

x_ = layers.Dropout(0.3)(x_)

x_ = layers.Flatten()(x_)

x_ = layers.Dense(204, activation="sigmoid")(x_)
outputs = layers.Reshape((68,3))(x_)

model = tf.keras.Model(inputs, outputs)

In [25]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
tf.math.truediv_4 (TFOpLambd (None, 244, 244, 3)       0         
_________________________________________________________________
tf.math.subtract_4 (TFOpLamb (None, 244, 244, 3)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 242, 242, 204)     5712      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 240, 240, 32)      58784     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 120, 120, 32)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 118, 118, 32)      9248

In [27]:
filepath = f"facial_model_{time.time()}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss="mean_absolute_percentage_error", optimizer="adam")
model.fit(x, y / 244, epochs=100000, batch_size=16, steps_per_epoch=1024, callbacks=callbacks_list, validation_data=(x_test, y_test / 244))

Epoch 1/100000
1024/1024 [==============================] - 154s 145ms/step - loss: 94.0498 - val_loss: 99.9943

Epoch 00001: val_loss improved from inf to 99.99435, saving model to facial_model_1637806965.7487626.h5
Epoch 2/100000
 127/1024 [==>...........................] - ETA: 2:09 - loss: 140.4763

KeyboardInterrupt: 

In [ ]:
filepath = f"facial_model_{time.time()}.h5"
model.save(filepath)

In [ ]:


# backbone = tf.keras.applications.MobileNetV2(
#     weights="imagenet", include_top=False, input_shape=(244,244,3)
# )

# backbone.trainable = False

inputs = layers.Input((244,244,3))
x_ = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x_ = layers.Conv2D(204, kernel_size=3, strides=1, activation="relu")(x_)
x_ = layers.Conv2D(
    64, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.MaxPooling2D()(x_)
x_ = layers.Conv2D(
    32, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.MaxPooling2D()(x_)
x_ = layers.Conv2D(
    32, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.MaxPooling2D()(x_)
x_ = layers.Conv2D(
    16, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.MaxPooling2D()(x_)
x_ = layers.Conv2D(
    16, kernel_size=3, strides=1, activation="relu"
)(x_)
x_ = layers.MaxPooling2D()(x_)

x_ = layers.UpSampling2D()(x_)
x_ = layers.TransposeConv2D(16, 3, 1, activation="relu")(x_)
x_ = layers.UpSampling2D()(x_)
x_ = layers.TransposeConv2D(16, 3, 1, activation="relu")(x_)
x_ = layers.UpSampling2D()(x_)
x_ = layers.TransposeConv2D(32, 3, 1, activation="relu")(x_)
x_ = layers.UpSampling2D()(x_)
x_ = layers.TransposeConv2D(32, 3, 1, activation="relu")(x_)
x_ = layers.UpSampling2D()(x_)
x_ = layers.TransposeConv2D(64, 3, 1, activation="relu")(x_)
x_ = layers.UpSampling2D()(x_)
outputs = layers.TransposeConv2D(204, 3, 1, activation="relu")(x_)





autoencoder = tf.keras.Model(inputs, outputs)